# Introduction

# Describe the problem and the source of data

# Import needed libraries

# Data Cleaning

## Explaning the features

## Describe the data statistically and visualize the data

## Checking for nulls values or outliers <- we can merge this with describing the data

## Preprocessing the data

# Models

## K-Nearest Neighbors

## Support Vector Machines

## Decision Tree

## Random Forest

## Voting Models -> ensemble learning

# Conclusion

## Compare the models